# Figure 11

Cloud water path histogram along the HALO flight track.

In [ ]:
import cmcrameri.cm as cmc
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
from lizard.mpltools import style
from lizard.writers.figure_to_file import write_figure

from si_clouds.io.readers.ancillary import read_ancillary_data
from si_clouds.io.readers.oem_result import read_oem_result_concat

In [ ]:
ds_anc = read_ancillary_data()

In [ ]:
ds_a, ds_op, _, _ = read_oem_result_concat(
    version="pub_r2_allsky_v1", test_id="", write=False
)

In [ ]:
# remove times where the retrieval was not valid (ze or 2m temperature)
ds_a = ds_a.sel(time=ds_anc.ix_retrieval_valid.sel(time=ds_a.time))
ds_op = ds_op.sel(time=ds_anc.ix_retrieval_valid.sel(time=ds_op.time))

In [ ]:
print("Number of retrievals", len(ds_a.time))
print("Mean convergence rate", ds_a.conv.mean().values)

In [ ]:
# get ancillary data for times where the retrieval converged
ds_anc = ds_anc.sel(time=ds_op.time)

In [ ]:
# get the indices from the ancillary dataset
data_hamp_all = [
    ds_op.cwp.sel(time=ds_anc.ix_all_liquid_0).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_all_liquid_1).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_all_liquid_2).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_all_no_liquid).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_all_potential_liquid).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_all_nodata).values * 1e3,
]

data_hamp_cea = [
    ds_op.cwp.sel(time=ds_anc.ix_cea_liquid_0).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_cea_liquid_1).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_cea_liquid_2).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_cea_no_liquid).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_cea_potential_liquid).values * 1e3,
    ds_op.cwp.sel(time=ds_anc.ix_cea_nodata).values * 1e3,
]

data_era5_all = [
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_liquid_0).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_liquid_1).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_liquid_2).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_no_liquid).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_potential_liquid).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_all_nodata).values * 1e3,
]

data_era5_cea = [
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_liquid_0).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_liquid_1).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_liquid_2).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_no_liquid).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_potential_liquid).values * 1e3,
    ds_anc.era5_tclw.sel(time=ds_anc.ix_cea_nodata).values * 1e3,
]

data_comb = [data_hamp_all, data_hamp_cea, data_era5_all, data_era5_cea]

color = np.concat(
    [
        cmc.batlow_r(np.linspace(0.1, 0.9, len(data_hamp_all) - 2)),
        cmc.grayC_r(np.linspace(0.2, 0.5, 2)),
    ]
)
label = [
    r"$h_{l}$ < 0.5 km",
    r"0.5 km $\leq$ $h_{l}$ < 1 km",
    r"1 km $\leq$ $h_{l}$ < 5 km",
    "No liq.",
    "Pot. liq.",
    "No data",
]

In [ ]:
bins = np.arange(0, 1000, 25)

fig, axes = plt.subplots(
    2,
    2,
    figsize=(5, 5),
    layout="constrained",
    sharex=True,
    sharey=True,
)

axes[0, 0].annotate(
    "(a) HAMP (all)",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="bottom",
)
axes[0, 1].annotate(
    "(b) ERA5 (all)",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="bottom",
)
axes[1, 0].annotate(
    f"(c) HAMP (Central Arctic)",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="bottom",
)
axes[1, 1].annotate(
    f"(d) ERA5 (Central Arctic)",
    xy=(0, 1),
    xycoords="axes fraction",
    ha="left",
    va="bottom",
)

for i, (ax, data) in enumerate(zip(axes.flat, data_comb)):
    if i == 0:
        kwds = dict(label=label)
    else:
        kwds = dict()
    ax.hist(
        data,
        bins=bins,
        histtype="bar",
        color=color,
        stacked=True,
        edgecolor="k",
        density=True,
        linewidth=0,
        **kwds,
    )

    # add inset axis zoom
    x0, y0, width, height = 0, 0, 400, 0.007
    axins = ax.inset_axes(
        [0.15, 0.35, 0.8, 0.6],
        xlim=(x0, x0 + width),
        ylim=(y0, y0 + height),
        yticklabels=[],
    )
    axins.hist(
        data,
        bins=bins,
        histtype="bar",
        color=color,
        stacked=True,
        edgecolor="k",
        density=True,
        linewidth=0,
    )
    ax.indicate_inset((x0, y0, width, height), edgecolor="k")

    axins.yaxis.set_major_locator(mticker.MultipleLocator(0.005))
    axins.yaxis.set_minor_locator(mticker.MultipleLocator(0.0025))
    axins.xaxis.set_major_locator(mticker.MultipleLocator(100))
    axins.xaxis.set_minor_locator(mticker.MultipleLocator(25))

    ax.yaxis.set_major_locator(mticker.MultipleLocator(0.005))
    ax.yaxis.set_minor_locator(mticker.MultipleLocator(0.0025))
    ax.xaxis.set_major_locator(mticker.MultipleLocator(100))
    ax.xaxis.set_minor_locator(mticker.MultipleLocator(25))
    ax.set_xlim(0, 500)


axes[1, 0].set_xlabel("CWP [g m$^{-2}$]")
axes[1, 1].set_xlabel("CWP [g m$^{-2}$]")
axes[1, 0].set_ylabel("Frequency")
axes[0, 0].set_ylabel("Frequency")

fig.legend(
    loc="lower center", bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=True
)

plt.show()

write_figure(
    fig,
    "paper/fig11.png",
    dpi=300,
    bbox_inches="tight",
)